---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    graph_id = []
    for G in P1_Graphs:
        avg_cluster = nx.average_clustering(G)
        max_degree = max(nx.degree(G).values())
        avg_path = nx.average_shortest_path_length(G)
        if (max_degree > 50):
            graph_id.append('PA')
        elif (avg_cluster < .1):
            graph_id.append('SW_H')
        else:
            graph_id.append('SW_L')
        #print(avg_cluster, max_degree, avg_path)
        #print('\n')
    
    return graph_id
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [52]:
def salary_predictions():
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    
    df = pd.DataFrame(index=G.nodes())
    df['Management'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Degree'] = pd.Series(G.degree())
    df['Cluster'] = pd.Series(nx.clustering(G))
    df['Close'] = pd.Series(nx.closeness_centrality(G))
    df['Between'] = pd.Series(nx.betweenness_centrality(G))
    
    X= df[['Degree', 'Between', 'Cluster', 'Close', 'Management']].dropna(axis=0)
    X=X[['Degree', 'Between', 'Cluster', 'Close']]
    y= df['Management'].dropna(axis=0)
    
    train_X, test_X, train_y, test_y = train_test_split(X,y, random_state=1, test_size=0.1)
    clf = LogisticRegression(C=100).fit(train_X, train_y)
    y_predict = clf.predict_proba(test_X)[:,1]
    auc_score = roc_auc_score(test_y, y_predict)
    
    null_df = df[df['Management'].isnull()]
    null_df = null_df[['Degree', 'Between', 'Cluster', 'Close']]
    manage_prob = pd.Series(clf.predict_proba(null_df)[:,1], index = null_df.index)
    
    
    return manage_prob


salary_predictions()

1      0.1024
2      0.6069
5      0.9480
8      0.1360
14     0.2619
18     0.2125
27     0.2667
30     0.4208
31     0.1817
34     0.1476
37     0.0711
40     0.1741
45     0.1473
54     0.3082
55     0.3046
60     0.1490
62     0.9940
65     0.4019
77     0.0686
79     0.1407
97     0.0837
101    0.0354
103    0.2441
108    0.1085
113    0.5240
122    0.0208
141    0.3664
142    0.9210
144    0.0641
145    0.5283
        ...  
913    0.0211
914    0.0602
915    0.0001
918    0.1523
923    0.0122
926    0.0760
931    0.0352
934    0.0039
939    0.0025
944    0.0002
945    0.0179
947    0.1801
950    0.0871
951    0.0162
953    0.0106
959    0.0003
962    0.0002
963    0.1428
968    0.0895
969    0.0883
974    0.0898
984    0.0033
987    0.1076
989    0.0957
991    0.1061
992    0.0005
994    0.0003
996    0.0002
1000   0.0303
1001   0.0859
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
def new_connections_predictions():
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    
    df = future_connections
    df['Jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['Pref Attach'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    
    
    X=df[['Jaccard', 'Pref Attach', 'Future Connection']].dropna(axis=0)
    X = X[['Jaccard', 'Pref Attach']]
    y= df['Future Connection'].dropna(axis=0)
    
    
    train_X, test_X, train_y, test_y = train_test_split(X, y)
    clf = LogisticRegression().fit(train_X, train_y)
    
    predict_prob = clf.predict_proba(test_X)[:,1]
    auc_score = roc_auc_score(test_y, predict_prob)
    
    df = df[df['Future Connection'].isnull()]
    X_null = df[['Jaccard', 'Pref Attach']]
    future_conn_prob = pd.Series(clf.predict_proba(X_null)[:,1], index = df.index)
    
    return future_conn_prob
    
new_connections_predictions()

(107, 348)    0.019866
(542, 751)    0.010660
(20, 426)     0.534622
(50, 989)     0.010408
(942, 986)    0.010144
(324, 857)    0.010442
(13, 710)     0.120696
(19, 271)     0.286786
(319, 878)    0.010322
(659, 707)    0.010634
(49, 843)     0.010245
(208, 893)    0.010573
(377, 469)    0.029904
(405, 999)    0.014105
(129, 740)    0.016329
(292, 618)    0.025442
(239, 689)    0.010305
(359, 373)    0.019274
(53, 523)     0.045713
(276, 984)    0.010258
(202, 997)    0.010207
(604, 619)    0.608039
(270, 911)    0.010309
(261, 481)    0.112269
(200, 450)    0.621447
(213, 634)    0.010740
(644, 735)    0.230924
(346, 553)    0.011408
(521, 738)    0.012838
(422, 953)    0.024140
                ...   
(672, 848)    0.010309
(28, 127)     0.688764
(202, 661)    0.011527
(54, 195)     0.999873
(295, 864)    0.010481
(814, 936)    0.010700
(839, 874)    0.010144
(139, 843)    0.010460
(461, 544)    0.013964
(68, 487)     0.014173
(622, 932)    0.010542
(504, 936)    0.027320
(479, 528) 